In [12]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

In [13]:
train = pd.read_csv('C:\\Users\\guilh_000\\Documents\\nlpdisaster\\train.csv')
test = pd.read_csv('C:\\Users\\guilh_000\\Documents\\nlpdisaster\\test.csv')

print(train.shape)
print(test.shape)

(7613, 5)
(3263, 4)


In [25]:
import string
import re
 
from nltk.corpus import stopwords 
stopwords_english = stopwords.words('english')
 
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
 
from nltk.tokenize import TweetTokenizer
 
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)
 
def clean_tweets(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
 
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
 
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
 
    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords_english and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
 
    return tweets_clean

In [28]:
train['normalized'] = train.text.apply(clean_tweets)
train['normal'] = [' '.join(map(str, l)) for l in train['normalized']]

In [29]:
train.head()

,id,keyword,location,text,target,normalized,normal
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"[deed, reason, earthquak, may, allah, forgiv, us]",deed reason earthquak may allah forgiv us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, rong, sask, canada]",forest fire near la rong sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[resid, ask, shelter, place, notifi, offic, ev...",resid ask shelter place notifi offic evacu she...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"[13,000, peopl, receiv, wildfir, evacu, order,...","13,000 peopl receiv wildfir evacu order califo..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"[got, sent, photo, rubi, alaska, smoke, wildfi...",got sent photo rubi alaska smoke wildfir pour ...


In [30]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 7 columns):
id            7613 non-null int64
keyword       7552 non-null object
location      5080 non-null object
text          7613 non-null object
target        7613 non-null int64
normalized    7613 non-null object
normal        7613 non-null object
dtypes: int64(2), object(5)
memory usage: 416.4+ KB


In [31]:
train = train.drop(['id', 'keyword', 'location'], axis=1)
train.head()

,text,target,normalized,normal
0,Our Deeds are the Reason of this #earthquake M...,1,"[deed, reason, earthquak, may, allah, forgiv, us]",deed reason earthquak may allah forgiv us
1,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, rong, sask, canada]",forest fire near la rong sask canada
2,All residents asked to 'shelter in place' are ...,1,"[resid, ask, shelter, place, notifi, offic, ev...",resid ask shelter place notifi offic evacu she...
3,"13,000 people receive #wildfires evacuation or...",1,"[13,000, peopl, receiv, wildfir, evacu, order,...","13,000 peopl receiv wildfir evacu order califo..."
4,Just got sent this photo from Ruby #Alaska as ...,1,"[got, sent, photo, rubi, alaska, smoke, wildfi...",got sent photo rubi alaska smoke wildfir pour ...


In [32]:
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer  

count_vectorizer = CountVectorizer(ngram_range=(1,2))
vectorized_data =count_vectorizer.fit_transform(train['normal'])

tfidfconverter = TfidfTransformer()  
tfid_data = tfidfconverter.fit_transform(vectorized_data)

indexed_data =hstack((np.array(range(0,tfid_data.shape[0]))[:,None],tfid_data))

#vectorizer = TfidfVectorizer(min_df= 3, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
#final_features = vectorizer.fit_transform(train['normal'])

#indexed_data =hstack((np.array(range(0,vectorized_data.shape[0]))[:,None],vectorized_data))
#indexed_data =hstack((np.array(range(0,final_features.shape[0]))[:,None],final_features))

#vectorized_data.shape
tfid_data.shape
#final_features.shape

#tfid resulted the better results for the competition testing dataset 

(7613, 54339)

In [33]:
targets = train.iloc[:, 1]

from sklearn.model_selection import train_test_split 
data_train, data_test, targets_train, targets_test =train_test_split(indexed_data, targets, test_size=0.25,random_state=100)
data_train_index = data_train[:,0]
data_train = data_train[:,1:]
data_test_index = data_test[:,0]
data_test = data_test[:,1:]

In [34]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(random_state=0)
lrc.fit(data_train, targets_train)

LogisticRegression(random_state=0)

In [36]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, classification_report

y_pred = lrc.predict(data_test)

print(confusion_matrix(targets_test, y_pred))
print(classification_report(targets_test, y_pred))
print(accuracy_score(targets_test, y_pred))

[[1050   59]
 [ 333  462]]
              precision    recall  f1-score   support

           0       0.76      0.95      0.84      1109
           1       0.89      0.58      0.70       795

    accuracy                           0.79      1904
   macro avg       0.82      0.76      0.77      1904
weighted avg       0.81      0.79      0.78      1904

0.7941176470588235


In [37]:
########
newtest = test
newtest.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [38]:
newtest = newtest.drop(['id', 'keyword', 'location'], axis=1)

In [40]:
newtest['normalized'] = newtest.text.apply(clean_tweets)
newtest['normal'] = [' '.join(map(str, l)) for l in newtest['normalized']]

In [41]:
newtest.head()

,text,normalized,normal
0,Just happened a terrible car crash,"[happen, terribl, car, crash]",happen terribl car crash
1,"Heard about #earthquake is different cities, s...","[heard, earthquak, differ, citi, stay, safe, e...",heard earthquak differ citi stay safe everyon
2,"there is a forest fire at spot pond, geese are...","[forest, fire, spot, pond, gees, flee, across,...",forest fire spot pond gees flee across street ...
3,Apocalypse lighting. #Spokane #wildfires,"[apocalyps, light, spokan, wildfir]",apocalyps light spokan wildfir
4,Typhoon Soudelor kills 28 in China and Taiwan,"[typhoon, soudelor, kill, 28, china, taiwan]",typhoon soudelor kill 28 china taiwan


In [44]:
tweet_to_predict = newtest['normal']
x = count_vectorizer.transform(tweet_to_predict)
xdata = tfidfconverter.transform(x)

In [52]:
def submit(preprocdata, subfile, model):
    #sub_df = pd.read_csv(subfile)
    subfile['target'] = model.predict(preprocdata)
    subfile.to_csv("C:\\Users\\guilh_000\\Documents\\nlpdisaster\\submission.csv", index = False, header=True)

In [48]:
test = test.drop(['keyword', 'location', 'text'], axis=1)
test.head()

,id
0,0
1,2
2,3
3,9
4,11


In [53]:
submit(xdata, test, lrc)